<a href="https://colab.research.google.com/github/savinthie/Final_Year_Project_IDP_2024-2025/blob/main/Model_8_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, Flatten, Dense, ReLU, Concatenate, MaxPooling1D,Dropout
from tensorflow.keras.optimizers import Adam
import joblib
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/FYP 2024 25/USDataset.csv', header=1)
df = df.fillna(0)

Mounted at /content/drive


In [ ]:
# Data Preprocessing
df.columns = [col.lower().replace(' ', '').replace('.', '') for col in df.columns]
cols_to_keep = ['stateabv', 'county', 'family', 'housing', 'food', 'transportation',
                'healthcare', 'othernecessities', 'childcare', 'taxes', 'total',
                'median_family_income', 'num_counties_in_st']
df1 = df[cols_to_keep].copy()

df1['median_family_income'] = df1['median_family_income'].replace(',', '', regex=True).astype(float)

In [ ]:
# Feature Engineering
df1['n_parents'] = df1['family'].str.slice(0, 1).astype(int)
df1['n_children'] = df1['family'].str.slice(2, 3).astype(int)
df1['n_members'] = df1['n_parents'] + df1['n_children']
df1['financial_stability'] = df1['median_family_income'] / df1['total']
df1["per_member_cost"] = df1["total"] / df1["n_members"]
df1["child_expense_cost"] = df1["per_member_cost"]*df1["n_children"]
df1["parent_expense_cost"] = df1["per_member_cost"]*df1["n_parents"]
df1["other_expense_cost"] = df1["total"] - (df1["child_expense_cost"]+df1["parent_expense_cost"])

In [ ]:
# Splitting the data
X = df1[['total', 'median_family_income', 'num_counties_in_st', 'n_children', 'n_parents', 'n_members']+['per_member_cost','child_expense_cost','parent_expense_cost','other_expense_cost']].values
y_expenses = df1[['housing', 'food', 'transportation', 'healthcare', 'othernecessities', 'childcare', 'taxes']].values

target_col_list = ['housing', 'food', 'transportation', 'healthcare', 'othernecessities', 'childcare', 'taxes']

# Scaling
scaler_X = StandardScaler()

# Load the X scaler
# scaler_X = joblib.load('scaler_X.pkl')

X_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()

# Load the y scaler
# scaler_y = joblib.load('scaler_y.pkl')
y_exp_scaled = scaler_y.fit_transform(y_expenses)



# Save the X scaler
joblib.dump(scaler_X, 'scaler_X.pkl')

# Save the y scaler
joblib.dump(scaler_y, 'scaler_y.pkl')

['scaler_y.pkl']

In [ ]:
# Cross-Validation Setup
kf = KFold(n_splits=5, shuffle=True, random_state=100)

# Metrics
def print_metrics(y_true, y_pred, task_name):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{task_name} - MSE: {mse:.4f}, R2: {r2:.4f}")

def regr_report(x, y):
    mae = round(mean_absolute_error(x, y), 4)
    r2 = round(r2_score(x, y), 4)
    mse = round(mean_squared_error(x, y), 4)
    rmse = round(np.sqrt(mean_squared_error(x, y)), 4)
    return f'MAE: {mae}, R-Squared: {r2}, RMSE: {rmse}, MSE: {mse}'

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, Dropout, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

# L2 regularization strength (you can tune this value)
l2_strength = 0.01

# Hybrid model setup
def create_hybrid_model(input_shape, output_shape):
    input_layer = Input(shape=input_shape)

    # CNN part with Dropout and L2 Regularization
    x = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu',
               kernel_regularizer=l2(l2_strength))(input_layer)  # L2 Regularization
    x = Dropout(0.2)(x)  # Dropout after the convolutional layer
    x = Flatten()(x)

    # ANN part with Dropout and L2 Regularization
    x = Dense(128, activation='relu', kernel_regularizer=l2(l2_strength))(x)
    x = Dropout(0.2)(x)  # Dropout after first dense layer
    x = Dense(64, activation='relu', kernel_regularizer=l2(l2_strength))(x)
    x = Dropout(0.3)(x)  # Dropout after second dense layer
    x = Dense(32, activation='relu', kernel_regularizer=l2(l2_strength))(x)
    x = Dropout(0.3)(x)  # Dropout after third dense layer

    # Output layers
    output_layers = [Dense(1, activation='linear', name=f'target_{col}')(x) for col in target_col_list]

    model = Model(inputs=input_layer, outputs=output_layers)

    return model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Cross-Validation with Evaluation and Training/Validation Loss Curve
def cross_val_with_evaluation(X_scaled, y_exp_scaled, kf, num_epochs=500):
    fold = 1

    # To store metrics across all folds
    metrics_summary = {target: {'train': [], 'val': [], 'test': []} for target in target_col_list}

    # To store loss values for each fold
    all_train_loss = []
    all_val_loss = []

    for train_index, val_index in kf.split(X_scaled):
        print(f"Fold {fold}/{kf.get_n_splits()}")
        X_train, X_test_val = X_scaled[train_index], X_scaled[val_index]
        y_train, y_test_val = y_exp_scaled[train_index], y_exp_scaled[val_index]

        # 50/50 split of remaining 20% for test and validation sets
        X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=100)

        # Reshaping data for CNN
        X_train = X_train[..., np.newaxis]
        X_val = X_val[..., np.newaxis]
        X_test = X_test[..., np.newaxis]

        # Early stopping to prevent overfitting
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=40,
            verbose=1,
            mode='min',
            restore_best_weights=True
        )

        # Create and compile the model
        model = create_hybrid_model((X_train.shape[1], 1), y_train.shape[1])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
        model.summary()

        # Training
        history = model.fit(
            X_train, [y_train[:, i] for i in range(y_train.shape[1])],
            validation_data=(X_val, [y_val[:, i] for i in range(y_val.shape[1])]),
            epochs=num_epochs,
            batch_size=128,
            verbose=1,
            callbacks=[early_stopping]
        )

        # Store loss values for the current fold
        all_train_loss.append(history.history['loss'])
        all_val_loss.append(history.history['val_loss'])

        # Predictions
        y_train_pred = np.column_stack(model.predict(X_train))
        y_val_pred = np.column_stack(model.predict(X_val))
        y_test_pred = np.column_stack(model.predict(X_test))  # Prediction on test set

        # Inverse scaling
        y_train_pred_original = scaler_y.inverse_transform(y_train_pred)
        y_train_original = scaler_y.inverse_transform(y_train)
        y_val_pred_original = scaler_y.inverse_transform(y_val_pred)
        y_val_original = scaler_y.inverse_transform(y_val)
        y_test_pred_original = scaler_y.inverse_transform(y_test_pred)  # Inverse transform for test set
        y_test_original = scaler_y.inverse_transform(y_test)  # Inverse transform for test set

        # Metrics for each target and accumulate results
        for i, target in enumerate(target_col_list):
            print(f'Model Results for {target.capitalize()} (Fold {fold}):')
            train_metrics = regr_report(y_train_original[:, i], y_train_pred_original[:, i])
            val_metrics = regr_report(y_val_original[:, i], y_val_pred_original[:, i])
            test_metrics = regr_report(y_test_original[:, i], y_test_pred_original[:, i])

            # Store metrics for this fold
            metrics_summary[target]['train'].append(train_metrics)
            metrics_summary[target]['val'].append(val_metrics)
            metrics_summary[target]['test'].append(test_metrics)

            print('Train Data:', train_metrics)
            print('Validation Data:', val_metrics)
            print('Test Data:', test_metrics)
            print()

        fold += 1

    # Compute and display final average metrics across all folds
    print("Final Average Metrics After 5 Folds:")
    for target in target_col_list:
        print(f"Results for {target.capitalize()}:")

        # Modified to calculate average metrics from numerical values
        train_metrics_df = pd.DataFrame([eval(x.replace('MAE: ', '').replace('R-Squared: ', '').replace('RMSE: ', '').replace('MSE: ', '').replace(', ', ',')) for x in metrics_summary[target]['train']], columns=['MAE', 'R-Squared', 'RMSE', 'MSE'])
        avg_train_metrics = train_metrics_df.mean().to_dict()

        val_metrics_df = pd.DataFrame([eval(x.replace('MAE: ', '').replace('R-Squared: ', '').replace('RMSE: ', '').replace('MSE: ', '').replace(', ', ',')) for x in metrics_summary[target]['val']], columns=['MAE', 'R-Squared', 'RMSE', 'MSE'])
        avg_val_metrics = val_metrics_df.mean().to_dict()

        test_metrics_df = pd.DataFrame([eval(x.replace('MAE: ', '').replace('R-Squared: ', '').replace('RMSE: ', '').replace('MSE: ', '').replace(', ', ',')) for x in metrics_summary[target]['test']], columns=['MAE', 'R-Squared', 'RMSE', 'MSE'])
        avg_test_metrics = test_metrics_df.mean().to_dict()

        print("Average Train Metrics:", avg_train_metrics)
        print("Average Validation Metrics:", avg_val_metrics)
        print("Average Test Metrics:", avg_test_metrics)
        print()

    # Plot Training and Validation Loss
    plt.figure(figsize=(10, 6))
    for i, (train_loss, val_loss) in enumerate(zip(all_train_loss, all_val_loss)):
        plt.plot(val_loss, label=f'Fold {i + 1} Validation Loss')
        plt.plot(train_loss, label=f'Fold {i + 1} Train Loss')


    plt.title("Training and Validation Loss Across Folds")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

    return model, y_train_original, y_train_pred_original, y_val_original, y_val_pred_original, y_test_original, y_test_pred_original

# Cross-Validation Execution
model, y_train_original, y_train_pred_original, y_val_original, y_val_pred_original, y_test_original, y_test_pred_original = cross_val_with_evaluation(X_scaled, y_exp_scaled, kf)
model.save_weights("model_expenses.weights.h5")


Fold 1/5


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10, 1)          │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 10, 8)          │             32 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 10, 8)          │              0 │ conv1d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 80)             │              0 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         10,368 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          8,256 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64)             │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 32)             │          2,080 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 32)             │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_housing (Dense)    │ (None, 1)              │             33 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_food (Dense)       │ (None, 1)              │             33 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_transportation     │ (None, 1)              │             33 │ dropout_3[0][0]        │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_healthcare (Dense) │ (None, 1)              │             33 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_othernecessities   │ (None, 1)              │             33 │ dropout_3[0][0]        │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_childcare (Dense)  │ (None, 1)              │             33 │ dropout_3[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ target_taxes (Dense)      │ (None, 1)              │             33 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 20,967 (81.90 KB)

 Trainable params: 20,967 (81.90 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
197/197 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6.0666 - target_childcare_loss: 0.6177 - target_food_loss: 0.4743 - target_healthcare_loss: 0.6479 - target_housing_loss: 0.6244 - target_othernecessities_loss: 0.5958 - target_taxes_loss: 0.5403 - target_transportation_loss: 0.7056

In [ ]:
def plot_residuals(y_true, y_pred, target_name):
    """
    Plot residuals (actual - predicted) for a specific target.
    """
    residuals = y_true - y_pred

    plt.figure(figsize=(8, 5))
    plt.scatter(y_pred, residuals, alpha=0.5, label="Residuals")
    plt.axhline(y=0, color='red', linestyle='--', label='Zero Error Line')
    plt.title(f"Residual Analysis for {target_name}")
    plt.xlabel("Predicted Values")
    plt.ylabel("Residuals")
    plt.legend()
    plt.grid(True)
    plt.show()

# Perform residual analysis for each target category
for i, target in enumerate(target_col_list):
    print(f"Residual Analysis for {target.capitalize()}:")

    # Training Residuals
    print("Training Data:")
    plot_residuals(
        y_train_original[:, i], y_train_pred_original[:, i], f"{target} (Training)"
    )

    # Validation Residuals
    print("Validation Data:")
    plot_residuals(
        y_val_original[:, i], y_val_pred_original[:, i], f"{target} (Validation)"
    )

    # Test Residuals
    print("Test Data:")
    plot_residuals(
        y_test_original[:, i], y_test_pred_original[:, i], f"{target} (Test)"
    )


In [ ]:
def get_prediction(model, input_data):
    input_data_val = input_data.values
    input_data_val_scaled = scaler_X.fit_transform(input_data_val)
    input_data_val_scaled = input_data_val_scaled[..., np.newaxis]
    input_data_pred = np.column_stack(model.predict(input_data_val_scaled))
    input_data_pred_original = scaler_y.inverse_transform(input_data_pred)
    return input_data_pred_original

In [ ]:
input_data = pd.DataFrame(
              [[7000,40000, 50, 2, 0, 1]],
              columns=['total', 'median_family_income', 'num_counties_in_st', 'n_children', 'n_parents', 'n_members']
        )

input_data["per_member_cost"] = input_data["total"] / input_data["n_members"]
input_data["child_expense_cost"] = input_data["per_member_cost"]*input_data["n_children"]
input_data["parent_expense_cost"] = input_data["per_member_cost"]*input_data["n_parents"]
input_data["other_expense_cost"] = input_data["total"] - (input_data["child_expense_cost"]+input_data["parent_expense_cost"])

In [ ]:
input_data_pred_original = get_prediction(model, input_data)

In [ ]:
data_row = input_data_pred_original[0]
data_row =np.array(input_data.values[0].tolist()+data_row.tolist())

In [ ]:
x_features = input_data[['total', 'median_family_income', 'num_counties_in_st', 'n_children', 'n_parents', 'n_members']].values[0]
x_features

In [ ]:
data_row = np.array(input_data["median_family_income"].values.tolist()+input_data_pred_original[0].tolist())

In [ ]:
data_row

In [ ]:
import pandas as pd
import numpy as np

input_data = pd.DataFrame(
    [[7000, 40000, 50, 2, 0, 1]],
    columns=['total', 'median_family_income', 'num_counties_in_st', 'n_children', 'n_parents', 'n_members']
)

input_data["per_member_cost"] = input_data["total"] / input_data["n_members"]
input_data["child_expense_cost"] = input_data["per_member_cost"] * input_data["n_children"]
input_data["parent_expense_cost"] = input_data["per_member_cost"] * input_data["n_parents"]
input_data["other_expense_cost"] = input_data["total"] - (input_data["child_expense_cost"] + input_data["parent_expense_cost"])

input_data_pred_original = get_prediction(model, input_data)


# Print predictions with labels
target_col_list = ['housing', 'food', 'transportation', 'healthcare', 'othernecessities', 'childcare', 'taxes']
for i, prediction in enumerate(input_data_pred_original[0]):
    print(f"{target_col_list[i]}: {prediction}")